In [67]:
import numpy as np
from matplotlib import pyplot as pl

import scipy.io.wavfile as wav
from scikits.talkbox.features import mfcc

%matplotlib inline

import os
BASE_DIR = 'vowels/'
files = os.listdir(BASE_DIR)

# 7.1

In [68]:
nafFiles = []
namFiles = []

# Get all natural female and male wav files by itering all the files' names.
# The first value of each array's elements is the simple rate, while the second one is the values.
for f in files:
    if f.startswith("naf"):
        nafFiles.append(wav.read(BASE_DIR + f))
    if f.startswith("nam"):
        namFiles.append(wav.read(BASE_DIR + f))

In [71]:
nafCeps = []
for f in nafFiles:
    nafCeps.append(mfcc(f[1], fs=f[0])[0])

namCeps = []
for f in namFiles:
    namCeps.append(mfcc(f[1], fs=f[0])[0])

In [78]:
nafMeanCeps = []
for c in nafCeps:
    nafMeanCeps.append(np.mean(c, axis=0))

In [79]:
namMeanCeps = []
for c in namCeps:
    namMeanCeps.append(np.mean(c, axis=0))

# 7.2

In [81]:
nkFiles = []

for f in files:
    if f.startswith("nk"):
        nkFiles.append(wav.read(BASE_DIR + f))

In [82]:
nkCeps = []
for f in nkFiles:
    nkCeps.append(mfcc(f[1], fs=f[0])[0])

In [83]:
nkMeanCeps = []
for c in nkCeps:
    nkMeanCeps.append(np.mean(c, axis=0))

# 7.3

# 7.4

# 7.5

In [49]:
naturalFiles = []
synthesizedFiles = []

for f in files:
    if "n" in f:
        naturalFiles.append(wav.read(BASE_DIR + f))
    if "s" in f:
        synthesizedFiles.append(wav.read(BASE_DIR + f))

In [52]:
naturalCeps = []
for f in naturalFiles:
    naturalCeps.append(mfcc(f[1], fs=f[0])[0])

synthesizedCeps = []
for f in synthesizedFiles:
    synthesizedCeps.append(mfcc(f[1], fs=f[0])[0])

In [85]:
naturalMeanCeps = []
for c in naturalCeps:
    naturalMeanCeps.append(np.mean(c, axis=0))

In [86]:
synthesizedMeanCeps = []
for c in synthesizedCeps:
    synthesizedMeanCeps.append(np.mean(c, axis=0))

# 7.6 7yo vs 3yo

In [56]:
_3yoFiles = []
_7yoFiles = []

for f in files:
    if f.startswith("nk3"):
        _3yoFiles.append(wav.read(BASE_DIR + f))
    if f.startswith("nk7"):
        _7yoFiles.append(wav.read(BASE_DIR + f))

In [57]:
_3yoCeps = []
for f in _3yoFiles:
    _3yoCeps.append(mfcc(f[1], fs=f[0])[0])

_7yoCeps = []
for f in _7yoFiles:
    _7yoCeps.append(mfcc(f[1], fs=f[0])[0])

In [59]:
_3yoMeanCeps = []
for c in naturalCeps:
    _3yoMeanCeps.append(np.mean(c, axis=0))

[ 16.79714949   1.54090856   0.11806946   0.75369388  -1.12860447
  -0.07059938   0.15398047   0.6023547   -0.39026959   0.66859702
   0.11442308  -0.02079662   0.25363452]


In [60]:
_7yoMeanCeps = np.mean(_7yoCeps[0], axis=0)
print _7yoMeanCeps

[  1.56494220e+01   1.81918608e+00  -6.28691171e-01  -6.50586184e-01
  -5.23806822e-01   7.55097353e-01   1.78484449e-01   3.07568984e-01
   9.78368565e-02  -2.11253999e-02   8.12625526e-02  -1.06741066e-02
   1.63699384e-01]
